In [ ]:
!ls /content/drive/MyDrive/

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
 agastya_project   'Graduate Docs'			       'VISA Folder'
'Colab Notebooks'  'Saved from the Google app'
 Credentials	    Statement_MAR2025_028949549.pdf.undefined


In [ ]:
import os

folder_path = '/content/drive/MyDrive/agastya_project'
os.chdir(folder_path)


In [ ]:
# 1. Unmount if needed
from google.colab import drive
drive.flush_and_unmount()

# 2. Clean mount point
import shutil
shutil.rmtree('/content/drive', ignore_errors=True)

# 3. Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# 4. Create project directories
from pathlib import Path
import os

PROJECT_ROOT = Path("/content/drive/MyDrive/agastya_project")
CACHE_DIR = PROJECT_ROOT / "data/raw/hf_cache"
OUT_DIR = PROJECT_ROOT / "data/processed"
EMBED_DIR = PROJECT_ROOT / "embeddings"
CHROMA_DIR = PROJECT_ROOT / "chromadb"

for d in [CACHE_DIR, OUT_DIR, CHROMA_DIR, EMBED_DIR]:
    os.makedirs(d, exist_ok=True)

print("✅ Google Drive mounted and directories created:")
!ls -R /content/drive/MyDrive/agastya_project

Mounted at /content/drive
✅ Google Drive mounted and directories created:
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/content/drive/MyDrive/agastya_project:
agastya_pipeline.py  cluster.py  embeddings		rag_generate.ipynb
chromadb	     data	 query_vector_db.ipynb	Untitled.ipynb

/content/drive/MyDrive/agastya_project/chromadb:

/content/drive/MyDrive/agastya_project/data:
processed  raw

/content/drive/MyDrive/agastya_project/data/processed:

/content/drive/MyDrive/agastya_project/data/raw:
hf_cache

/content/drive/MyDrive/agastya_project/data/raw/hf_cache:

/content/drive/MyDrive/agastya_project/embeddings:


In [ ]:
import os
import re
import json
import logging
import unicodedata
from typing import List, Dict, Any
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from huggingface_hub import hf_hub_download

# ============== CONFIGURATION ==============
PROJECT_ROOT = Path("/content/drive/MyDrive/agastya_project")
CACHE_DIR = PROJECT_ROOT / "data/raw/hf_cache"
OUT_DIR = PROJECT_ROOT / "data/processed"
EMBED_DIR = PROJECT_ROOT / "embeddings"
CHROMA_DIR = PROJECT_ROOT / "chromadb"

for d in [CACHE_DIR, OUT_DIR, CHROMA_DIR, EMBED_DIR]:
    os.makedirs(d, exist_ok=True)

DATA_FILES = {
    "train_en": "train.en.csv",
    "train_sn": "train.sn.csv",
    "dev_en":   "dev.en.csv",
    "dev_sn":   "dev.sn.csv",
    "test_en":  "test.en.csv",
    "test_sn":  "test.sn.csv",
}

DANDA, DOUBLE_DANDA = "।", "॥"

# ============== LOGGING ==============
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ============== REGEX PRECOMPILATION ==============
PAT_DOUBLE_DANDA = re.compile(r"\|\|")
PAT_SINGLE_DANDA = re.compile(r"(?<!\|)\|(?!\|)")
PAT_WHITESPACE = re.compile(r"\s+")
PAT_SN_REMOVE = re.compile(r"\[\*?\d*\]")
PAT_SN_PAREN = re.compile(r"\(\s*—[^)]*\)")

# ============== TEXT NORMALIZATION HELPERS ==============
def normalize_unicode(s: str) -> str:
    """Normalize Unicode characters for Sanskrit text."""
    return unicodedata.normalize("NFC", s or "")

def normalize_danda(s: str) -> str:
    """Normalize Sanskrit punctuation."""
    if not isinstance(s, str):
        return ""
    s = normalize_unicode(s)
    s = PAT_DOUBLE_DANDA.sub(DOUBLE_DANDA, s)
    s = PAT_SINGLE_DANDA.sub(DANDA, s)
    s = s.replace("।।", DOUBLE_DANDA)
    s = PAT_WHITESPACE.sub(" ", s).strip()
    return s

def clean_sn(s: str) -> str:
    """Clean Sanskrit text by removing footnotes and unwanted patterns."""
    s = normalize_danda(s or "")
    s = PAT_SN_REMOVE.sub("", s)
    s = PAT_SN_PAREN.sub("", s)
    s = PAT_WHITESPACE.sub(" ", s).strip()
    return s

def clean_en(s: str) -> str:
    """Clean English text by normalizing whitespace."""
    return PAT_WHITESPACE.sub(" ", (s or "")).strip()

# ============== FILE HELPERS ==============
def _download(filename: str) -> str:
    """Download file from HuggingFace Hub."""
    return hf_hub_download(
        repo_id="rahular/itihasa",
        repo_type="dataset",
        filename=filename,
        local_dir=str(CACHE_DIR),
        local_dir_use_symlinks=False
    )

def _read_lines(path: str) -> List[str]:
    """Read lines from a file with UTF-8 encoding."""
    return Path(path).read_text(encoding="utf-8-sig").splitlines()

# ============== DATA PROCESSING ==============
def _pair_split(en_file: str, sn_file: str, split_name: str) -> pd.DataFrame:
    """Create a DataFrame pairing Sanskrit and English lines for a given split."""
    en_path = _download(en_file)
    sn_path = _download(sn_file)
    en_lines = _read_lines(en_path)
    sn_lines = _read_lines(sn_path)
    if len(en_lines) != len(sn_lines):
        raise ValueError(
            f"[{split_name}] line count mismatch: EN={len(en_lines)} vs SN={len(sn_lines)}\n"
            f"EN file: {en_path}\nSN file: {sn_path}\n"
            "Try deleting the cache and rerun."
        )
    df = pd.DataFrame({"sn": sn_lines, "en": en_lines})
    df["split"] = split_name
    return df

def load_flatten_from_repo() -> pd.DataFrame:
    logging.info("▶️ Downloading split files...")
    train = _pair_split(DATA_FILES["train_en"], DATA_FILES["train_sn"], "train")
    dev = _pair_split(DATA_FILES["dev_en"], DATA_FILES["dev_sn"], "validation")
    test = _pair_split(DATA_FILES["test_en"], DATA_FILES["test_sn"], "test")
    df = pd.concat([train, dev, test], ignore_index=True)
    logging.info("▶️ Cleaning Sanskrit & English text...")
    df["sn"] = df["sn"].map(clean_sn)
    df["en"] = df["en"].map(clean_en)
    df["text"] = (df["sn"] + "\n\n" + df["en"]).str.strip()
    by_split = df["split"].value_counts().to_dict()
    logging.info(f"✔ Loaded rows by split: {by_split}")
    return df

def make_windows(df: pd.DataFrame, k: int = 2) -> pd.DataFrame:
    logging.info(f"▶️ Building {k}-verse windows...")
    out = []
    for split, g in df.groupby("split", sort=False):
        g = g.reset_index(drop=True)
        for i in range(len(g) - k + 1):
            sn = " ".join(g.loc[i:i + k - 1, "sn"])
            en = " ".join(g.loc[i:i + k - 1, "en"])
            out.append({"split": split, "sn": sn, "en": en, "text": f"{sn}\n\n{en}"})
    win = pd.DataFrame(out)
    logging.info(f"✔ Windowed rows: {len(win)}")
    return win

def save_parquet_and_jsonl(df: pd.DataFrame, stem: str):
    pq_path = OUT_DIR / f"{stem}.parquet"
    df.to_parquet(pq_path, index=False)
    logging.info(f"💾 Parquet → {pq_path}")
    jsonl_path = OUT_DIR / f"{stem}.jsonl"
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for rec in df.to_dict("records"):
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    logging.info(f"💾 JSONL   → {jsonl_path}")

# ========== EMBEDDING, VECTOR DB, AND CLUSTERING MODULES ==========

from sentence_transformers import SentenceTransformer
import chromadb
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np

### CONFIG
EMBEDDING_MODEL = "BAAI/bge-large-en-v1.5"    # Strong accurate model (downloaded automatically)
CHUNK_SIZE = 256
CHUNK_OVERLAP = 32
BATCH_SIZE = 4
COLLECTION_NAME = "itihasa_sanskrit"
EMBEDDING_DIM = 1024  # For "bge-large-en-v1.5"
PCA_DIM = 50

class SanskritTextSplitter:
    def __init__(self, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separators = [
            "\n\n", "।।", "॥", "।", "|", ".", "?", "!", ";", ",", " ", ""
        ]
    def split_text(self, text: str) -> List[str]:
        chunks = self._recursive_split(text, self.separators)
        return self._add_overlap(chunks)
    def _recursive_split(self, text: str, separators: List[str]) -> List[str]:
        if not separators:
            return [text[i:i + self.chunk_size] for i in range(0, len(text), self.chunk_size)]
        sep, rest = separators[0], separators[1:]
        splits = text.split(sep) if sep else list(text)
        chunks, current = [], ""
        for i, split in enumerate(splits):
            if sep and i < len(splits) - 1:
                split += sep
            if len(current) + len(split) <= self.chunk_size:
                current += split
            else:
                if current: chunks.append(current.strip())
                if len(split) > self.chunk_size:
                    chunks.extend(self._recursive_split(split, rest))
                    current = ""
                else:
                    current = split
        if current: chunks.append(current.strip())
        return [c for c in chunks if c]
    def _add_overlap(self, chunks: List[str]) -> List[str]:
        if len(chunks) <= 1 or self.chunk_overlap == 0: return chunks
        overlapped = [chunks[0]]
        for i in range(1, len(chunks)):
            overlap = chunks[i - 1][-self.chunk_overlap:]
            overlapped.append(overlap + " " + chunks[i])
        return overlapped
    def create_documents(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        logging.info("\n--- STEP: Text Splitting ---")
        docs = []
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            combined = f"Sanskrit: {row['sn']}\n\nEnglish: {row['en']}"
            chunks = self.split_text(combined)
            for chunk_idx, chunk in enumerate(chunks):
                docs.append({
                    "text": chunk,
                    "metadata": {
                        "verse_id": idx,
                        "chunk_id": chunk_idx,
                        "split": row["split"],
                        "sanskrit_original": row["sn"],
                        "english_original": row["en"],
                        "total_chunks": len(chunks)
                    }
                })
        logging.info(f"✔ Created {len(docs)} document chunks")
        return docs

class EmbeddingGenerator:
    def __init__(self): self.model = None
    def load_model(self):
        logging.info(f"--- STEP: Loading Embedding Model ({EMBEDDING_MODEL}) ---")
        self.model = SentenceTransformer(EMBEDDING_MODEL)
    def generate_embeddings(self, docs: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        if self.model is None: self.load_model()
        texts = [doc["text"] for doc in docs]
        logging.info(f"Generating embeddings for {len(texts)} chunks...")
        all_embeddings = []
        for i in tqdm(range(0, len(texts), BATCH_SIZE)):
            batch_texts = texts[i:i + BATCH_SIZE]
            batch_embeds = self.model.encode(
                batch_texts,
                batch_size=BATCH_SIZE,
                normalize_embeddings=True,
                show_progress_bar=False
            )
            all_embeddings.extend(batch_embeds.tolist())
        for doc, embedding in zip(docs, all_embeddings):
            doc["embedding"] = embedding
        logging.info(f"✔ Generated {len(all_embeddings)} embeddings")
        return docs


In [ ]:

class ChromaDBManager:
    def __init__(self, reset_collection=False):
        self.client = chromadb.PersistentClient(path=str(CHROMA_DIR))
        if reset_collection:
            try:
                self.client.delete_collection(name=COLLECTION_NAME)
            except: pass
        self.collection = self.client.get_or_create_collection(
            name=COLLECTION_NAME,
            metadata={"hnsw:space": "cosine", "embedding_dim": str(EMBEDDING_DIM)}
        )
    def ingest_documents(self, docs: List[Dict[str, Any]]):
        logging.info("--- STEP: Ingesting to ChromaDB ---")
        ids, embeddings, texts, metadatas = [], [], [], []
        for doc in docs:
            doc_id = f"verse_{doc['metadata']['verse_id']}_chunk_{doc['metadata']['chunk_id']}"
            ids.append(doc_id)
            embeddings.append(doc["embedding"])
            texts.append(doc["text"])
            meta = {
                "verse_id": int(doc["metadata"]["verse_id"]),
                "chunk_id": int(doc["metadata"]["chunk_id"]),
                "split": doc["metadata"]["split"],
                "total_chunks": int(doc["metadata"]["total_chunks"])
            }
            metadatas.append(meta)
        for i in tqdm(range(0, len(ids), max(1, BATCH_SIZE // 2))):
            sl = slice(i, i + max(1, BATCH_SIZE // 2))
            self.collection.add(
                 ids=ids[sl],
                 embeddings=embeddings[sl],
                 documents=texts[sl],
                 metadatas=metadatas[sl]
                )
        logging.info(f"✔ Ingested {len(ids)} documents")
    def search(self, query: str, n_results: int = 5):
        embed_gen = EmbeddingGenerator()
        embed_gen.load_model()
        q_emb = embed_gen.model.encode([query], normalize_embeddings=True)[0].tolist()
        results = self.collection.query(query_embeddings=[q_emb], n_results=n_results)
        return results

class LangChainTester:
    def test_retrieval(self):
        logging.info("--- STEP: LangChain Sanity Test ---")
        embedding_function = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
        vectorstore = Chroma(
            collection_name=COLLECTION_NAME,
            embedding_function=embedding_function,
            persist_directory=str(CHROMA_DIR)
        )
        retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        test_queries = [
            "Tell me about Agastya in Ramayana and Mahabharata",
            "What does the text say about family of Agastya?",
            "Describe the role of Agastya"
        ]
        for query in test_queries:
            logging.info(f"\nQuery: {query}")
            results = retriever.invoke(query)
            for i, doc in enumerate(results, 1):
                logging.info(f"  Result {i}: {doc.page_content[:200]}...  Meta: {doc.metadata}")
        logging.info("✔ LangChain test completed.")

### === CLUSTERING STAGE (Advanced) ===
def reduce_embeddings(docs_emb, out_dims=PCA_DIM):
    embeds = np.array([doc['embedding'] for doc in docs_emb if 'embedding' in doc])
    pca = PCA(n_components=out_dims)
    reduced = pca.fit_transform(embeds)
    for i, doc in enumerate(docs_emb):
        doc['embedding_reduced'] = reduced[i]
    logging.info(f"✔ Embeddings reduced to {out_dims} dimensions.")
    return docs_emb, reduced

def cluster_and_evaluate(embeddings, min_k=3, max_k=8):
    results = []
    for k in range(min_k, max_k+1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=15).fit(embeddings)
        sil = silhouette_score(embeddings, kmeans.labels_)
        db = davies_bouldin_score(embeddings, kmeans.labels_)
        ch = calinski_harabasz_score(embeddings, kmeans.labels_)
        logging.info(f"k={k}: Silhouette={sil:.3f}, DB={db:.3f}, CH={ch:.3f}")
        results.append((k, sil, db, ch, list(kmeans.labels_)))
    return results

def print_sampled_clusters(docs, labels, num_samples=3):
    from collections import defaultdict
    clusters = defaultdict(list)
    for doc, label in zip(docs, labels):
        clusters[label].append(doc['text'])
    for label, texts in clusters.items():
        logging.info(f"\n--- Cluster {label} ---")
        for t in texts[:num_samples]:
            logging.info(f"• {t[:200].replace(chr(10), ' ')}")

### ======= MAIN PIPELINE =======
def main():
    logging.info('============ AGASTYA END-TO-END RAG PIPELINE FULL ============')
    df = load_flatten_from_repo()
    save_parquet_and_jsonl(df, "itihasa_chunks_atomic_full")
    win2 = make_windows(df, k=2)
    save_parquet_and_jsonl(win2, "itihasa_chunks_win2_full")

    splitter = SanskritTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    docs = splitter.create_documents(df)

    emb_gen = EmbeddingGenerator()
    docs_emb = emb_gen.generate_embeddings(docs)

    # Advanced clustering: Dim. reduction + k sweep + reporting
    docs_emb, reduced = reduce_embeddings(docs_emb, out_dims=PCA_DIM)
    metrics = cluster_and_evaluate(reduced, min_k=3, max_k=8)
    chosen_k = 5  # Pick best from printed metrics or automate
    chosen_labels = next(r for r in metrics if r[0]==chosen_k)[4]
    print_sampled_clusters(docs_emb, chosen_labels, num_samples=2)

    m_df = pd.DataFrame([(r[0], r[1], r[2], r[3]) for r in metrics],
                    columns=['k','Silhouette','Davies-Bouldin','Calinski-Harabasz'])
    m_df.to_csv(str(OUT_DIR/'clustering_metrics.csv'), index=False)

    chroma_mgr = ChromaDBManager(reset_collection=False)
    chroma_mgr.ingest_documents(docs_emb)
    tester = LangChainTester()
    tester.test_retrieval()
    logging.info('✅ Pipeline complete.')

if __name__ == "__main__":
    main()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_downloa

KeyboardInterrupt: 

In [ ]:
!ls -lh /content/drive/MyDrive/agastya_project/agastya_project/data/processed/

total 0


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires l

In [ ]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 21.7 MB/s

In [ ]:
!pip install chromadb

In [ ]:
!pip install matplotlib

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import numpy as np # Import numpy

# Load embedding model
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Load ChromaDB
vectorstore = Chroma(
    collection_name="itihasa_sanskrit",
    embedding_function=embedding_function,
    persist_directory="/content/drive/MyDrive/agastya_project/agastya_project/chromadb"
)

# Check if the ChromaDB collection has documents
try:
    collection_count = vectorstore._collection.count()
    print(f"ChromaDB collection count: {collection_count}")
    if collection_count == 0:
        print("Warning: ChromaDB collection is empty. No documents to retrieve.")
except Exception as e:
    print(f"Error checking ChromaDB collection count: {e}")
    print("This might indicate that the ChromaDB was not properly initialized or persisted.")

# Define queries
queries = ["Agastya", "Agastya Muni", "Sage Agastya", "Rishi Agastya", "Kumbhayoni"]

# Retrieve and deduplicate documents
retrieved_docs = []
seen_texts = set()
for query in queries:
    docs = vectorstore.similarity_search(query, k=3)
    for doc in docs:
        if doc.page_content not in seen_texts:
            retrieved_docs.append(doc)
            seen_texts.add(doc.page_content)

# Check if any documents were retrieved
if not retrieved_docs:
    print("\nError: No documents were retrieved for the given queries. Cannot perform clustering.")
    print("Please check the queries or the content of the ChromaDB.")
else:
    texts = [doc.page_content for doc in retrieved_docs]
    embeddings = embedding_function.embed_documents(texts)

    # Convert embeddings to a NumPy array
    embeddings = np.array(embeddings)

    # Check if embeddings array is still empty after processing
    if embeddings.size == 0:
        print("\nError: Embeddings array is empty despite retrieving documents. This indicates an issue with embedding generation.")
    else:
        # KMeans clustering
        # Ensure n_clusters is not greater than the number of samples, and at least 2 samples for clustering
        n_samples = len(embeddings)
        if n_samples < 2:
            print(f"\nWarning: Not enough samples ({n_samples}) for clustering. Skipping KMeans and Hierarchical Clustering.")
            kmeans_labels = np.zeros(n_samples) # Assign all to one cluster if not enough for multiple
        else:
            n_clusters_kmeans = min(5, n_samples)
            kmeans = KMeans(n_clusters=n_clusters_kmeans, random_state=42, n_init=10)
            kmeans_labels = kmeans.fit_predict(embeddings)

            # Hierarchical clustering
            linkage_matrix = linkage(embeddings, method='ward')

            # PCA visualization
            if n_samples > 1:
                pca = PCA(n_components=2)
                pca_result = pca.fit_transform(embeddings)
                plt.figure(figsize=(8, 6))
                plt.scatter(pca_result[:, 0], pca_result[:, 1], c=kmeans_labels, cmap='tab10')
                plt.title("KMeans Clustering (PCA)")
                plt.xlabel("PCA 1")
                plt.ylabel("PCA 2")
                plt.show()
            else:
                print(f"\nWarning: Not enough samples ({n_samples}) for PCA visualization. Skipping PCA.")

            # t-SNE visualization
            if n_samples > 1:
                tsne_perplexity = min(5, n_samples - 1)
                tsne = TSNE(n_components=2, random_state=42, perplexity=tsne_perplexity)
                tsne_result = tsne.fit_transform(embeddings)
                plt.figure(figsize=(8, 6))
                plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=kmeans_labels, cmap='tab10')
                plt.title("KMeans Clustering (t-SNE)")
                plt.xlabel("t-SNE 1")
                plt.ylabel("t-SNE 2")
                plt.show()
            else:
                print(f"\nWarning: Not enough samples ({n_samples}) for t-SNE visualization. Skipping t-SNE.")

            # Dendrogram for hierarchical clustering
            if n_samples > 1:
                plt.figure(figsize=(10, 6))
                dendrogram(linkage_matrix, labels=[f"Doc {i}" for i in range(len(texts))])
                plt.title("Hierarchical Clustering Dendrogram")
                plt.xlabel("Documents")
                plt.ylabel("Distance")
                plt.show()
            else:
                print(f"\nWarning: Not enough samples ({n_samples}) for Dendrogram visualization. Skipping Dendrogram.")

            # If clustering was performed, calculate metrics (ensure enough samples)
            if n_samples >= 2:
                from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

                # Ensure that there are enough unique labels for silhouette score (at least 2 unique labels)
                if len(np.unique(kmeans_labels)) > 1:
                    sil_score = silhouette_score(embeddings, kmeans_labels)
                    db_score = davies_bouldin_score(embeddings, kmeans_labels)
                    ch_score = calinski_harabasz_score(embeddings, kmeans_labels)

                    print(f"\nSilhouette Score: {sil_score}")
                    print(f"Davies-Bouldin Index: {db_score}")
                    print(f"Calinski-Harabasz Score: {ch_score}")
                else:
                    print("\nWarning: Not enough unique cluster labels (or samples) to calculate clustering metrics.")
            else:
                print("\nWarning: Not enough samples to calculate clustering metrics.")

/tmp/ipython-input-1412769916.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
print(f"Embedding array shape: {np.array(embeddings).shape}")



Embedding array shape: (0,)


In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

sil_score = silhouette_score(embeddings, kmeans_labels)
db_score = davies_bouldin_score(embeddings, kmeans_labels)
ch_score = calinski_harabasz_score(embeddings, kmeans_labels)

print(f"Silhouette Score: {sil_score}")
print(f"Davies-Bouldin Index: {db_score}")
print(f"Calinski-Harabasz Score: {ch_score}")


Silhouette Score: 0.08059181467437945
Davies-Bouldin Index: 2.426642682454497
Calinski-Harabasz Score: 3.72327557894088
